In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c4be3f9-cb8d-44ec-b065-70bc667ccc92;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 538ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [2]:
df = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://catarina-3sprint-sptech-bucket-cliente/randon/dfrandon.csv')

22/06/15 13:41:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show()

+------------------------------------------------------+
|ID,LITROS_GASTOS,QUANTIDADE_AMOSTRAS,PH,LITROS_AMOSTRA|
+------------------------------------------------------+
|                                  1,5000050000,0,0....|
|                                  2,20000100000,14,...|
|                                  3,45000150000,15,...|
|                                  4,80000200000,54,...|
|                                  5,"1,25E+11",55,0...|
|                                  6,5000050000,83,0...|
|                                  7,20000100000,83,...|
|                                  8,45000150000,84,...|
|                                  9,80000200000,37,...|
|                                  10,"1,25E+11",38,...|
|                                  11,5000050000,38,...|
|                                  12,20000100000,38...|
|                                  13,45000150000,39...|
|                                  14,80000200000,34...|
|                              

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

#_df = df.select(
#    col('ID').alias('ID'),
#    col('LITROS_GASTOS').alias('LITROS_GASTOSS'),
#    col('QUANTIDADE_AMOSTRAS').alias('QUANTIDADE_AMOSTRAS'),
#    col('PH').alias('PH')
#    col('LITROS_AMOSTRA').alias('amostraEmLitros')      
#)


# _df.write \
#    .mode('overwrite') \
#    .csv('s3a://marise-3cco/vendas_anuais')

df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://catarina-sprint3.database.windows.net;databaseName=catarina-sprint3;') \
 .option('dbtable', 'DFRANDON') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()